In [5]:
import numpy as np
import pandas as pd
import random
import os
import logging
from collections import defaultdict
from tqdm import trange
import json
from datetime import datetime

In [6]:
class SimulatedAnnealingSupplyChainOptimizer:
    def __init__(self, demand_df, prices_df, max_load=8000, transport_cost=4500,
                 iterations=1000, initial_temp=10000, cooling_rate=0.995):

        self.demand_df = demand_df.copy()
        self.prices_df = prices_df.copy()
        self.max_load = max_load
        self.transport_cost = transport_cost
        self.iterations = iterations
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate

        self.demands = self.demand_df.groupby(['polygon', 'specie'])['demand'].sum().reset_index()
        self.species = self.demands['specie'].unique()
        self.suppliers = self.prices_df['supplier'].unique()

        self.log_file = 'SA_Supply_Chain.log'
        if os.path.exists(self.log_file):
            logging.shutdown()
            os.remove(self.log_file)
        logging.basicConfig(filename=self.log_file, level=logging.INFO)

    def initial_solution(self):
        remaining = self.demands.copy()
        order_schedule = []
        day = 0

        while remaining['demand'].sum() > 0:
            day += 1
            remaining_capacity = self.max_load
            fulfilled_indices = []
            day_plan = []

            for idx, row in remaining[remaining['demand'] > 0].sample(frac=1).iterrows():
                polygon, specie, demand = row['polygon'], row['specie'], row['demand']
                if demand > remaining_capacity:
                    continue

                valid_suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].unique()
                if len(valid_suppliers) == 0:
                    continue

                supplier = random.choice(valid_suppliers)

                day_plan.append((day, polygon, specie, supplier, demand))
                remaining_capacity -= demand
                fulfilled_indices.append(idx)

            order_schedule.extend(day_plan)
            remaining.loc[fulfilled_indices, 'demand'] = 0

        return order_schedule

    def fitness(self, schedule):
        cost = 0
        horizon_days = max(entry[0] for entry in schedule)
        for day in range(1, horizon_days + 1):
            day_orders = [entry for entry in schedule if entry[0] == day]
            day_suppliers = set()
            for (_, polygon, specie, supplier, amount) in day_orders:
                row = self.prices_df[(self.prices_df['specie'] == specie) &
                                     (self.prices_df['supplier'] == supplier)]
                unit_price = row.iloc[0]['price']
                cost += unit_price * amount
                day_suppliers.add(supplier)
            cost += self.transport_cost * len(day_suppliers)
        return cost

    def neighbor(self, schedule):
        new_schedule = schedule.copy()
        if len(new_schedule) < 2:
            return new_schedule

        idx = random.randint(0, len(new_schedule) - 1)
        day, polygon, specie, _, amount = new_schedule[idx]

        valid_suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].unique()
        new_supplier = random.choice(valid_suppliers)

        new_schedule[idx] = (day, polygon, specie, new_supplier, amount)
        return new_schedule

    def run(self):
        current_solution = self.initial_solution()
        current_cost = self.fitness(current_solution)
        best_solution = current_solution
        best_cost = current_cost

        temp = self.initial_temp

        for i in trange(self.iterations, desc='Simulated Annealing'):
            new_solution = self.neighbor(current_solution)
            new_cost = self.fitness(new_solution)

            if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temp):
                current_solution = new_solution
                current_cost = new_cost
                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost

            temp *= self.cooling_rate
            logging.info(f"Iteration {i+1}: Cost = {current_cost:.2f}, Temp = {temp:.2f}")

        logging.info(f"Best solution found with cost: {best_cost:.2f}")
        return best_solution, best_cost

In [ ]:
# Example usage:
demand_df = pd.read_csv('../setup/demand.csv').sample(frac=1)
prices_df = pd.read_csv('../setup/supplier_prices.csv')
optimizer = SimulatedAnnealingSupplyChainOptimizer(demand_df, prices_df, iterations=10_000)
best_solution, best_cost = optimizer.run()

temp = pd.DataFrame(best_solution, columns = ['Day', 'Polygon', 'Specie', 'Supplier', 'Amount'])
# temp.to_csv("../scheduling/SA_orders.csv", index=False)

Simulated Annealing: 100%|██████████| 1000/1000 [00:54<00:00, 18.48it/s]


In [9]:
best_cost

999860.0

In [8]:
temp

,Day,Polygon,Specie,Supplier,Amount
0,1,19,Opuntia streptacanta,Venado,251
1,1,1,Opuntia robusta,Laguna seca,313
2,1,17,Opuntia streptacanta,Venado,311
3,1,23,Agave scabra,Laguna seca,182
4,1,19,Yucca filifera,Vivero,103
...,...,...,...,...,...
125,5,23,Opuntia streptacanta,Venado,282
126,5,18,Prosopis laevigata,Vivero,490
127,5,5,Opuntia streptacanta,Venado,385
128,5,4,Agave scabra,Moctezuma,264
